In [3]:
import fxcmpy
import socketio
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sympy import *
init_printing()
import xlrd
import itertools
import datetime
from itertools import combinations
from itertools import product
from itertools import groupby
from astropy.table import QTable, Table, Column
from astropy import units as u
from IPython.display import HTML
import requests
import json

In [2]:
TOKEN = 'b199bb5a15fa26957990a3fb823a51bd473cca55'
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error', server='demo', log_file='log.txt')

pair = 'EUR/USD'
S = con.get_candles(pair, period='H1', number=2000)          ## SETTING TO 12 DAYS FOR NOW
S['open'] = (S['bidopen']+S['askopen'])/2
S['close'] = (S['bidclose']+S['askclose'])/2
S['high'] = (S['bidhigh']+S['askhigh'])/2
S['low'] = (S['bidlow']+S['asklow'])/2
S = S.assign(MOVE=(S['close'] - S['open']))
AvgCandle = np.average(abs(S['MOVE']))

s = 96                                      ## NUMBER OF 1-HOUR CANDLESTICKS WE WANT TO FORECAST AHEAD (4 DAYS)

CP = S['close'].iloc[-1]
l = int(S['close'].count())
Sd = np.linspace(1,l,num=l)

D = con.get_candles(pair, period='H4', number=500)          ## SETTING TO 12 DAYS FOR NOW
D['open'] = (D['bidopen']+D['askopen'])/2
D['close'] = (D['bidclose']+D['askclose'])/2
D['high'] = (D['bidhigh']+D['askhigh'])/2
D['low'] = (D['bidlow']+D['asklow'])/2
D = D.assign(MOVE=(D['close'] - D['open']))
AvgCandleH4 = np.average(abs(D['MOVE']))

k = int(D['close'].count())
Sdk = np.linspace(1,k,num=k)

In [3]:
## ICHIMOKU ANALYSIS SECTOR


p = k
RED = []
BLUE = []
GREEN = []
CLOUDA = []
CLOUDB = []

for x in Sdk:
    if p < 9:
        RLhigh = 1
        RLlow = 1
    else:
        RL = D[p-9:p]
        RLhigh = max(RL['high'])
        RLlow = min(RL['low'])
    redline = (RLhigh + RLlow)/2
    RED.append(redline)

    if p < 26:
        BLhigh = 1
        BLlow = 1
    else:
        BL = D[p-26:p]
        BLhigh = max(BL['high'])
        BLlow = min(BL['low'])
    blueline = (BLhigh + BLlow)/2
    BLUE.append(blueline)

    greenline = BL['close'][0]
    GREEN.append(greenline)

    if p < 52:
        CLhigh = 1
        CLlow = 1
    else:
        CL = D[p-52:p]
        CLhigh = max(CL['high'])
        CLlow = min(CL['low'])
    clouda = (redline+ blueline)/2
    cloudb = (CLhigh + CLlow)/2
    CLOUDA.append(clouda)
    CLOUDB.append(cloudb)

    p = p - 1

CLOUDA.reverse() 
CLOUDB.reverse() 
cloud = [i / j for i, j in zip(CLOUDA, CLOUDB)]
cloud = [x-1 for x in cloud]
D['Cloud'] = cloud
pclouda = D['close']/CLOUDA - 1
D['PvCloudA'] = pclouda
pcloudb = D['close']/CLOUDB - 1
D['PvCloudB'] = pcloudb

RED.reverse()
BLUE.reverse() 
redblue = [i / j for i, j in zip(RED, BLUE)]
redblue = [x-1 for x in redblue]
D['RedvBlue'] = redblue
pblue = D['close']/BLUE - 1
D['PvBlue'] = pblue

GREEN.reverse() 
pgreen = D['close']/GREEN - 1
D['PvGreen'] = pgreen

D['BLUE'] = BLUE
D = D.assign(BluevLast=((D['BLUE'] - D['BLUE'].shift())/D['BLUE']))

tg = int(round(k*(14/16),0))
trendnow = np.average(D['BluevLast'][tg::])                   ##        CURRENT TREND    !!!


if trendnow > 0:
    Igrade_cloud = cloud[-1]*100
    Igrade_pclouda = pclouda[-1]*100
    Igrade_pcloudb = pcloudb[-1]*100
    Igrade_redblue = redblue[-1]*100
    Igrade_pblue = pblue[-1]*100
    Igrade_pgreen = pgreen[-1]*100
    Igrade_blue = D['BluevLast'][-1]*100

elif trendnow < 0:
    Igrade_cloud = cloud[-1]*-100
    Igrade_pclouda = pclouda[-1]*-100
    Igrade_pcloudb = pcloudb[-1]*-100
    Igrade_redblue = redblue[-1]*-100
    Igrade_pblue = pblue[-1]*-100
    Igrade_pgreen = pgreen[-1]*-100
    Igrade_blue = D['BluevLast'][-1]*-100
    
if Igrade_cloud <= 0:
    Igrade_cloud = 0
if Igrade_pclouda <= 0:
    Igrade_pclouda = 0
if Igrade_pcloudb <= 0:
    Igrade_pcloudb = 0
if Igrade_redblue <= 0:
    Igrade_redblue = 0
if Igrade_pblue <= 0:
    Igrade_pblue = 0
if Igrade_pgreen <= 0:
    Igrade_pgreen = 0
if Igrade_blue <= 0:
    Igrade_blue = 0
    

K = list(D['Cloud'][k-52::])

Kgains = []
Klosses = []
sum_gains = 0
sum_losses = 1

if trendnow > 0:
    for x in K:
        if x > 0:
            Kgains.append(x)
        elif x < 0:
            Klosses.append(x)

        sum_gains = np.sum(Kgains)
        sum_losses = abs(np.sum(Klosses))
elif trendnow < 0:
    for x in K:
        if x < 0:
            Kgains.append(x)
        elif x > 0:
            Klosses.append(x)

        sum_gains = abs(np.sum(Kgains))
        sum_losses = abs(np.sum(Klosses))

Igrade_cloud2 = (sum_gains/(sum_gains+sum_losses))


K = list(D['PvCloudA'][k-52::])

Kgains = []
Klosses = []
sum_gains = 0
sum_losses = 1

if trendnow > 0:
    for x in K:
        if x > 0:
            Kgains.append(x)
        elif x < 0:
            Klosses.append(x)

        sum_gains = np.sum(Kgains)
        sum_losses = abs(np.sum(Klosses))
elif trendnow < 0:
    for x in K:
        if x < 0:
            Kgains.append(x)
        elif x > 0:
            Klosses.append(x)

        sum_gains = abs(np.sum(Kgains))
        sum_losses = abs(np.sum(Klosses))

Igrade_pclouda2 = (sum_gains/(sum_gains+sum_losses))


K = list(D['PvCloudB'][k-52::])

Kgains = []
Klosses = []
sum_gains = 0
sum_losses = 1

if trendnow > 0:
    for x in K:
        if x > 0:
            Kgains.append(x)
        elif x < 0:
            Klosses.append(x)

        sum_gains = np.sum(Kgains)
        sum_losses = abs(np.sum(Klosses))
elif trendnow < 0:
    for x in K:
        if x < 0:
            Kgains.append(x)
        elif x > 0:
            Klosses.append(x)

        sum_gains = abs(np.sum(Kgains))
        sum_losses = abs(np.sum(Klosses))

Igrade_pcloudb2 = (sum_gains/(sum_gains+sum_losses))


K = list(D['RedvBlue'][k-26::])

Kgains = []
Klosses = []
sum_gains = 0
sum_losses = 1

if trendnow > 0:
    for x in K:
        if x > 0:
            Kgains.append(x)
        elif x < 0:
            Klosses.append(x)

        sum_gains = np.sum(Kgains)
        sum_losses = abs(np.sum(Klosses))
elif trendnow < 0:
    for x in K:
        if x < 0:
            Kgains.append(x)
        elif x > 0:
            Klosses.append(x)

        sum_gains = abs(np.sum(Kgains))
        sum_losses = abs(np.sum(Klosses))

Igrade_redblue2 = (sum_gains/(sum_gains+sum_losses))

K = list(D['PvBlue'][k-26::])

Kgains = []
Klosses = []
sum_gains = 0
sum_losses = 1

if trendnow > 0:
    for x in K:
        if x > 0:
            Kgains.append(x)
        elif x < 0:
            Klosses.append(x)

        sum_gains = np.sum(Kgains)
        sum_losses = abs(np.sum(Klosses))
elif trendnow < 0:
    for x in K:
        if x < 0:
            Kgains.append(x)
        elif x > 0:
            Klosses.append(x)

        sum_gains = abs(np.sum(Kgains))
        sum_losses = abs(np.sum(Klosses))

Igrade_pblue2 = (sum_gains/(sum_gains+sum_losses))

K = list(D['PvGreen'][k-26::])

Kgains = []
Klosses = []
sum_gains = 0
sum_losses = 1

if trendnow > 0:
    for x in K:
        if x > 0:
            Kgains.append(x)
        elif x < 0:
            Klosses.append(x)

        sum_gains = np.sum(Kgains)
        sum_losses = abs(np.sum(Klosses))
elif trendnow < 0:
    for x in K:
        if x < 0:
            Kgains.append(x)
        elif x > 0:
            Klosses.append(x)

        sum_gains = abs(np.sum(Kgains))
        sum_losses = abs(np.sum(Klosses))

Igrade_pgreen2 = (sum_gains/(sum_gains+sum_losses))

K = list(D['BluevLast'][k-26::])

Kgains = []
Klosses = []
sum_gains = 0
sum_losses = 1

if trendnow > 0:
    for x in K:
        if x > 0:
            Kgains.append(x)
        elif x < 0:
            Klosses.append(x)

        sum_gains = np.sum(Kgains)
        sum_losses = abs(np.sum(Klosses))
elif trendnow < 0:
    for x in K:
        if x < 0:
            Kgains.append(x)
        elif x > 0:
            Klosses.append(x)

        sum_gains = abs(np.sum(Kgains))
        sum_losses = abs(np.sum(Klosses))

Igrade_blue2 = (sum_gains/(sum_gains+sum_losses))

Iw1 = 0.20
Iw2 = 0.20
Iw3 = 0.12
Iw4 = 0.12
Iw5 = 0.06
Iw6 = 0.30

Igrade = Iw1*Igrade_cloud + Iw2*((Igrade_pclouda + Igrade_pcloudb)/2) + Iw3*Igrade_redblue + Iw4*Igrade_pblue + Iw5*Igrade_pgreen + Iw6*Igrade_blue

Igrade2 = Iw1*Igrade_cloud2 + Iw2*((Igrade_pclouda2 + Igrade_pcloudb2)/2) + Iw3*Igrade_redblue2 + Iw4*Igrade_pblue2 + Iw5*Igrade_pgreen2 + Iw6*Igrade_blue2


In [4]:
## ICHIMOKU ACCURACY SECTOR

a = 0
f = 52
b = int(a + f + (s/4))
Sdk2 = Sdk[1:(k-b)]
J = []

for q in Sdk2:
    y = np.average(list(D['Cloud'][a:a+f]))
    Yprice = (D['close'][b]-CP)
    y = y/abs(y)
    Yprice = Yprice/abs(Yprice)
    if y == Yprice:
        j = 1
    else:
        j = 0
    J.append(j)
    a = a + 1

Iavg_cloud = np.average(J)


a = 0
f = 52
b = int(a + f + (s/4))
Sdk2 = Sdk[1:(k-b)]
J = []

for q in Sdk2:
    y = np.average(list(D['PvCloudA'][a:a+f]))
    Yprice = (D['close'][b]-CP)
    y = y/abs(y)
    Yprice = Yprice/abs(Yprice)
    if y == Yprice:
        j = 1
    else:
        j = 0
    J.append(j)
    a = a + 1

Iavg_pclouda = np.average(J)


a = 0
f = 52
b = int(a + f + (s/4))
Sdk2 = Sdk[1:(k-b)]
J = []

for q in Sdk2:
    y = np.average(list(D['PvCloudB'][a:a+f]))
    Yprice = (D['close'][b]-CP)
    y = y/abs(y)
    Yprice = Yprice/abs(Yprice)
    if y == Yprice:
        j = 1
    else:
        j = 0
    J.append(j)
    a = a + 1

Iavg_pcloudb = np.average(J)


a = 0
f = 52
b = int(a + f + (s/4))
Sdk2 = Sdk[1:(k-b)]
J = []

for q in Sdk2:
    y = np.average(list(D['RedvBlue'][a:a+f]))
    Yprice = (D['close'][b]-CP)
    y = y/abs(y)
    Yprice = Yprice/abs(Yprice)
    if y == Yprice:
        j = 1
    else:
        j = 0
    J.append(j)
    a = a + 1

Iavg_redblue = np.average(J)


a = 0
f = 52
b = int(a + f + (s/4))
Sdk2 = Sdk[1:(k-b)]
J = []

for q in Sdk2:
    y = np.average(list(D['PvBlue'][a:a+f]))
    Yprice = (D['close'][b]-CP)
    y = y/abs(y)
    Yprice = Yprice/abs(Yprice)
    if y == Yprice:
        j = 1
    else:
        j = 0
    J.append(j)
    a = a + 1

Iavg_pblue = np.average(J)


a = 0
f = 52
b = int(a + f + (s/4))
Sdk2 = Sdk[1:(k-b)]
J = []

for q in Sdk2:
    y = np.average(list(D['PvGreen'][a:a+f]))
    Yprice = (D['close'][b]-CP)
    y = y/abs(y)
    Yprice = Yprice/abs(Yprice)
    if y == Yprice:
        j = 1
    else:
        j = 0
    J.append(j)
    a = a + 1

Iavg_pgreen = np.average(J)


a = 0
f = 52
b = int(a + f + (s/4))
Sdk2 = Sdk[1:(k-b)]
J = []

for q in Sdk2:
    y = np.average(list(D['BluevLast'][a:a+f]))
    Yprice = (D['close'][b]-CP)
    y = y/abs(y)
    Yprice = Yprice/abs(Yprice)
    if y == Yprice:
        j = 1
    else:
        j = 0
    J.append(j)
    a = a + 1

Iavg_blue = np.average(J)

Iavg = Iavg_cloud + Iavg_pclouda + Iavg_pcloudb + Iavg_redblue + Iavg_pblue + Iavg_pgreen + Iavg_blue

In [5]:
## TREND ANALYSIS SECTOR


M = []
H = []
G = []
a = l
b = 0

for q in Sd:
    x = np.linspace(1, a, num=a)
    y = S.iloc[b:,10]
    
    m = sm.OLS(y, x).fit()
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    m = slope
    h = intercept
    g = m*(a+s)+h

    M.append(m)
    H.append(h)
    G.append(g)
    
    a = a - 1
    b = b + 1

S['RPP'] = G               ## REGRESSION-PREDICTED PRICE   -    PRICE FORECASTED BY REGRESSION
G1 = G-CP




## TREND LINE CONFLUENCE GRADE - LONG TERM

w1 = int(round(l*(1/16),0))
M_long1 = np.average(M[w1:l-1])
w2 = int(round(l*(3/16),0))
M_long2 = np.average(M[w2:l-1])
w3 = int(round(l*(6/16),0))
M_long3 = np.average(M[w3:l-1])
Tgrade_long = (0.15*M_long1 + 0.35*M_long2 + 0.50*M_long3)
if trendnow > 0:
    if Tgrade_long > 0:
        Tgrade_long = Tgrade_long
    else:
        Tgrade_long = 0
elif trendnow < 0:
    if Tgrade_long > 0:
        Tgrade_long = 0
    else:
        Tgrade_long = -1*Tgrade_long
else:
    Tgrade_long = 0


## TREND LINE CONFLUENCE GRADE - MID TERM

w1 = int(round(l*(8/16),0))
M_mid1 = np.average(M[w1:l-1])
w2 = int(round(l*(10/16),0))
M_mid2 = np.average(M[w2:l-1])
w3 = int(round(l*(12/16),0))
M_mid3 = np.average(M[w3:l-1])
Tgrade_mid = (0.20*M_mid1 + 0.4*M_mid2 + 0.4*M_mid3)
if trendnow > 0:
    if Tgrade_mid > 0:
        Tgrade_mid = Tgrade_mid
    else:
        Tgrade_mid = 0
elif trendnow < 0:
    if Tgrade_mid > 0:
        Tgrade_mid = 0
    else:
        Tgrade_mid = -1*Tgrade_mid
else:
    Tgrade_mid = 0
    

## TREND LINE CONFLUENCE GRADE - SHORT TERM

w1 = int(round(l*(13/16),0))
M_short1 = np.average(M[w1:l-1])
w2 = int(round(l*(14/16),0))
M_short2 = np.average(M[w2:l-1])
w3 = int(round(l*(15/16),0))
M_short3 = np.average(M[w3:l-1])
Tgrade_short = (0.3*M_short1 + 0.3*M_short2 + 0.4*M_short3)
if trendnow > 0:
    if Tgrade_short > 0:
        Tgrade_short = Tgrade_short
    else:
        Tgrade_short = 0
elif trendnow < 0:
    if Tgrade_short > 0:
        Tgrade_short = 0
    else:
        Tgrade_short = -1*Tgrade_short
else:
    Tgrade_short = 0
    

## TREND LINE CONFLUENCE GRADE - FINAL

wlong = 0.15
wmid = 0.35
wshort = 0.50

Tgrade = (wlong*Tgrade_long) + (wmid*Tgrade_mid) + (wshort*Tgrade_short)




## TREND LINE PROJECTION - LONG TERM

w1 = int(round(l*(1/16),0))
G_long1 = np.average(G[w1:l-1])
w2 = int(round(l*(3/16),0))
G_long2 = np.average(G[w2:l-1])
w3 = int(round(l*(6/16),0))
G_long3 = np.average(G[w3:l-1])
Tline_long = (0.15*G_long1 + 0.35*G_long2 + 0.50*G_long3)


## TREND LINE PROJECTION - MID TERM

w1 = int(round(l*(8/16),0))
G_mid1 = np.average(G[w1:l-1])
w2 = int(round(l*(10/16),0))
G_mid2 = np.average(G[w2:l-1])
w3 = int(round(l*(12/16),0))
G_mid3 = np.average(G[w3:l-1])
Tline_mid = (0.20*G_mid1 + 0.4*G_mid2 + 0.4*G_mid3)


## TREND LINE PROJECTION - SHORT TERM

w1 = int(round(l*(13/16),0))
G_short1 = np.average(G[w1:l-1])
w2 = int(round(l*(14/16),0))
G_short2 = np.average(G[w2:l-1])
w3 = int(round(l*(15/16),0))
G_short3 = np.average(G[w3:l-1])
Tline_short = (0.3*G_short1 + 0.3*G_short2 + 0.4*G_short3)


## TREND LINE PROJECTION - FINAL

wlong = 0.15
wmid = 0.35
wshort = 0.50

Tline = (wlong*Tline_long) + (wmid*Tline_mid) + (wshort*Tline_short)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:140: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:142: RuntimeWarning: invalid value encountered in double_scalar

In [6]:
## TREND ACCURACY SECTOR


## ACCURACY OF 6-WEEK BLOCKS
    
a = 0
fa = 720
Sd2 = Sd[0:l-(fa+s)]
J = []

for q in Sd2:
    x = np.linspace((1+a), (fa+a), num=fa)
    y = S.iloc[a:(a+fa),10]
    
    m1 = sm.OLS(y, x).fit()
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

    m1 = slope
    h1 = intercept
    g1 = m1*(a+fa+s)+h1

    j = g1 - S.iloc[(a+fa+s),10]

    J.append(j)
    
    a = a + 1
    

J6W = np.average(J)


## ACCURACY OF 4-WEEK BLOCKS
    
a = 0
fb = 480
Sd2 = Sd[0:l-(fb+s)]
J = []

for q in Sd2:
    x = np.linspace((1+a), (fb+a), num=fb)
    y = S.iloc[a:(a+fb),10]
    
    m1 = sm.OLS(y, x).fit()
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

    m1 = slope
    h1 = intercept
    g1 = m1*(a+fb+s)+h1

    j = g1 - S.iloc[(a+fb+s),10]

    J.append(j)
    
    a = a + 1
    
J4W = np.average(J)


## ACCURACY OF 2-WEEK BLOCKS
    
a = 0
fc = 240
Sd2 = Sd[0:l-(fc+s)]
J = []

for q in Sd2:
    x = np.linspace((1+a), (fc+a), num=fc)
    y = S.iloc[a:(a+fc),10]
    
    m1 = sm.OLS(y, x).fit()
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

    m1 = slope
    h1 = intercept
    g1 = m1*(a+fc+s)+h1

    j = g1 - S.iloc[(a+fc+s),10]

    J.append(j)
    
    a = a + 1
    
J2W = np.average(J)

Javg = (0.25*J6W + 0.5*J4W + 0.25*J2W)


JTline = Tline + Javg

In [7]:
## RELATIVE STRENGTH INDEX SECTOR


T = D.close.diff()
window = 14
up_days = T.copy()
up_days[T<=0]=0.0
down_days = abs(T.copy())
down_days[T>0]=0.0
RS_up = up_days.rolling(window).mean()
RS_down = down_days.rolling(window).mean()
RSI= 100-100/(1+RS_up/RS_down)

Rgrade = RSI[-1]

if trendnow > 0:
    Rgrade = (100 - Rgrade)
else:
    Rgrade = Rgrade

In [8]:
## FINAL GRADE SECTOR



Iacc = Iavg
Tacc = abs(Javg)

I1w = 0.3
I2w = 0.3
Tw = 0.3
Rw = 0.1

Grade = int(I1w*Igrade*Iacc + I2w*Igrade2*Iacc + Tw*Tgrade + Rw*Rgrade)

if trendnow > 0:
    move = 'rise'
else:
    move = 'fall'

print('HERCULES predicts price of  ',pair,' will  ',move,'  to  ',JTline,'  after  ',s,'  hours. With a confidence level of  ',Grade)

HERCULES predicts price of   EUR/USD  will   fall   to   1.0734920211027088   after   96   hours. With a confidence level of   4
